In [13]:
!unzip -o data.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


# Module Tiền Xử Lý Dữ Liệu 

## Quy Trình Xử Lý Dữ Liệu Môi Trường Đa Thành Phố

### Mục Tiêu

1. **Tích Hợp Dữ Liệu**: Kết hợp bộ dữ liệu chất lượng không khí và thời tiết cho từng thành phố dựa trên căn chỉnh thời gian
2. **Kiểm Soát Chất Lượng**: Xác định và xử lý các mẫu dữ liệu bất thường bao gồm dữ liệu cảm biến đóng băng
3. **Điền Khuyết Dữ Liệu**: Áp dụng chiến lược điền khuyết kết hợp nội suy thời gian và k-láng giềng gần nhất
4. **Kỹ Thuật Đặc Trưng**: Tạo các đặc trưng dẫn xuất để cải thiện hiệu suất mô hình
5. **Chuẩn Hóa Dữ Liệu**: Đảm bảo tính nhất quán giữa các bộ dữ liệu đa thành phố

---

### Kiến Trúc Quy Trình Xử Lý

```
BƯỚC 1: Import Thư Viện
BƯỚC 2: Cấu Hình & Tham Số
BƯỚC 3: Định Nghĩa Hàm Hỗ Trợ
BƯỚC 4: Tải Dữ Liệu & Xử Lý Khởi Tạo
BƯỚC 5: Căn Chỉnh Thời Gian & Kết Hợp
BƯỚC 6: Kỹ Thuật Đặc Trưng
BƯỚC 7: Điền Khuyết Giá Trị
BƯỚC 8: Kiểm Định Chất Lượng & Xuất Dữ Liệu
BƯỚC 9: Tích Hợp Quy Trình Hoàn Chỉnh
BƯỚC 10: Thực Thi Xử Lý Hàng Loạt
BƯỚC 11: Thống Kê Tổng Hợp & Báo Cáo
```

---

### Nguồn Dữ Liệu

**Các Thành Phố Nghiên Cứu:**
- Cần Thơ (CT) - Khu vực đồng bằng sông Cửu Long
- Thành phố Hồ Chí Minh (HCM) - Trung tâm kinh tế phía Nam
- Hà Nội (HN) - Thủ đô phía Bắc

**Phạm Vi Thời Gian:** Tháng 1/2023 - Tháng 12/2025 (độ phân giải theo giờ)

**Thông Số Chất Lượng Không Khí:** AQI, CO, NO₂, O₃, PM10, PM2.5, SO₂

**Thông Số Khí Tượng:** Nhiệt độ, Độ ẩm tương đối, Áp suất, Tốc độ gió, Hướng gió, Độ che phủ mây, Lượng mưa, Chu kỳ ngày, Điểm sương

---

##  1: Import Thư Viện



In [1]:
# Thư viện xử lý dữ liệu cốt lõi
import numpy as np
import pandas as pd

# Học máy cho điền khuyết nâng cao
from sklearn.impute import KNNImputer

# Hàm tiện ích
import datetime
import warnings
warnings.filterwarnings('ignore')


---

##  2: Cấu Hình & Tham Số Toàn Cục

### Đặc Tả Đặc Trưng

Cấu hình này xác định:
- **Đặc Trưng Thời Tiết**: Chín biến số khí tượng thu thập từ các trạm thời tiết
- **Đặc Trưng Chất Lượng Không Khí**: Bảy phép đo chất ô nhiễm từ cảm biến môi trường
- **Khung Thời Gian**: Chuỗi thời gian theo giờ hoàn chỉnh từ 2023-01-01 đến 2025-12-01
- **Ánh Xạ Thành Phố**: Tên thư mục sang mã thành phố chuẩn hóa

### Lý Do

Khung thời gian đảm bảo chỉ mục thời gian đồng nhất trên tất cả các thành phố, tạo điều kiện cho phân tích so sánh và huấn luyện mô hình. Khoảng thời gian ba năm nắm bắt các biến động theo mùa và xu hướng dài hạn.

In [2]:
# Bộ đặc trưng khí tượng
feature_weather = ['temp', 'rh', 'pres', 'wind_spd', 'wind_dir', 'clouds', 'precip', 'pod', 'dewpt']

# Bộ chất ô nhiễm không khí
feature_air = ['aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2']

# Ánh xạ thành phố: tên thư mục -> mã chuẩn
cities_config = {
    'Cantho': 'CT',
    'HCM': 'HCM',
    'Hanoi': 'HN'
}

# Tạo chỉ mục thời gian đầy đủ theo độ phân giải giờ
full_time_index = pd.date_range(
    start="2023-01-01 00:00:00",
    end="2025-12-01 00:00:00",
    freq='h'
)

# Hiển thị tóm tắt cấu hình
print("TÓM TẮT CẤU HÌNH:")
print(f"   Đặc trưng thời tiết: {len(feature_weather)} biến số")
print(f"   Đặc trưng chất lượng không khí: {len(feature_air)} chất ô nhiễm")
print(f"   Thành phố phân tích: {list(cities_config.keys())}")
print(f"   Phạm vi thời gian: {full_time_index[0]} to {full_time_index[-1]}")
print(f"   Tổng số quan sát: {len(full_time_index):,} bản ghi theo giờ")

TÓM TẮT CẤU HÌNH:
   Đặc trưng thời tiết: 9 biến số
   Đặc trưng chất lượng không khí: 7 chất ô nhiễm
   Thành phố phân tích: ['Cantho', 'HCM', 'Hanoi']
   Phạm vi thời gian: 2023-01-01 00:00:00 to 2025-12-01 00:00:00
   Tổng số quan sát: 25,561 bản ghi theo giờ


---

##  3: Định Nghĩa Hàm Hỗ Trợ

### Hàm 1: remove_frozen_days()

**Mục Đích**: Phát hiện và loại bỏ dữ liệu cảm biến đóng băng (giá trị không đổi trong khoảng thời gian 24 giờ)

**Thuật Toán**:
1. Nhóm dữ liệu theo ngày lịch
2. Tính độ lệch chuẩn cho mỗi biến trong mỗi ngày
3. Xác định các ngày có std = 0 (không có biến động)
4. Thay thế giá trị đóng băng bằng NaN để điền khuyết sau này

**Loại Trừ**: Các biến 'pod' (phân loại) và 'precip' (tự nhiên bằng 0 trong thời kỳ khô) được loại trừ khỏi phát hiện dữ liệu đóng băng.

### Hàm 2: load_city_data()

**Mục Đích**: Tải các file CSV thô cho một thành phố với xử lý lỗi

**Tham Số**:
- city_folder: Tên thư mục chứa các file dữ liệu của thành phố
- city_code: Mã định danh thành phố chuẩn hóa (CT, HCM, HN)

**Giá Trị Trả Về**: Tuple của (air_quality_df, weather_df) hoặc (None, None) nếu tải thất bại

In [3]:
def remove_frozen_days(df, cols):
    """
    Phát hiện và loại bỏ dữ liệu cảm biến đóng băng.

    Dữ liệu đóng băng xảy ra khi cảm biến báo cáo cùng một giá trị liên tục
    trong cả ngày, cho thấy khả năng hỏng hóc hoặc lỗi truyền thông.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame đầu vào với chỉ mục datetime
    cols : list
        Tên cột để kiểm tra giá trị đóng băng

    Returns:
    --------
    pd.DataFrame
        DataFrame với giá trị đóng băng được thay bằng NaN
    """
    df_out = df.copy()
    df_out['temp_date'] = df_out.index.date

    print(f"  FROZEN DATA DETECTION: Analyzing {len(cols)} biến số...")
    total_frozen = 0

    for col in cols:
        if col in df_out.columns:
            # Tính độ lệch chuẩn theo ngày
            daily_std = df_out.groupby('temp_date')[col].transform('std')
            mask_frozen = (daily_std == 0)

            # Skip biến số that can legitimately be constant
            if col not in ['pod', 'precip']:
                frozen_count = mask_frozen.sum()
                if frozen_count > 0:
                    df_out.loc[mask_frozen, col] = np.nan
                    total_frozen += frozen_count
                    print(f"     {col}: {frozen_count} quan sát đóng băng được phát hiện")

    print(f"  RESULT: {total_frozen} tổng giá trị đóng băng được đánh dấu để điền khuyết")
    return df_out.drop(columns=['temp_date'])

def remove_physical_outliers(df, city_code):
    """
    Loại bỏ các giá trị vi phạm giới hạn vật lý (Hard Thresholds).
    Các giá trị này được coi là lỗi cảm biến và thay thế bằng NaN để điền khuyết sau.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dữ liệu đầu vào
    city_code : str
        Mã thành phố (để log)
        
    Returns:
    --------
    pd.DataFrame
        Dữ liệu đã lọc bỏ giá trị vô lý
    """
    df_out = df.copy()
    
    # ĐỊNH NGHĨA NGƯỠNG CHO VIỆT NAM (Based on Domain Knowledge)
    # Cấu trúc: 'tên_cột': (min_chấp_nhận, max_chấp_nhận)
    thresholds = {
        # Chất lượng không khí (µg/m³)
        'pm25': (0, 600),   # 600 là ngưỡng rất cao, vượt qua là lỗi
        'pm10': (0, 800),   
        'aqi':  (0, 500),   # AQI max thường là 500
        'co':   (0, 30000), # CO đo bằng ppb hoặc µg/m3, cần check unit. Giả sử µg/m3
        'no2':  (0, 1000),
        'so2':  (0, 1000),
        'o3':   (0, 800),
        
        # Thời tiết
        'temp': (5, 45),     # Nhiệt độ VN hiếm khi < 5 hoặc > 45
        'rh':   (0, 100),    # Độ ẩm max 100%
        'pres': (950, 1050), # Áp suất khí quyển (hPa)
        'wind_spd': (0, 50), # Tốc độ gió (m/s), 50m/s là siêu bão cấp 15-16
        'clouds': (0, 100),  # Mây 0-100%
        'precip': (0, 200)   # Mưa mm/h
    }

    print(f"  PHYSICAL OUTLIER DETECTION: Kiểm tra các giá trị phi thực tế...")
    total_removed = 0
    
    for col, (min_val, max_val) in thresholds.items():
        if col in df_out.columns:
            # Tìm các giá trị vi phạm
            mask_outlier = (df_out[col] < min_val) | (df_out[col] > max_val)
            count = mask_outlier.sum()
            
            if count > 0:
                # Thay thế bằng NaN
                df_out.loc[mask_outlier, col] = np.nan
                total_removed += count
                print(f"     {col}: Đã loại bỏ {count} mẫu (Ngoài vùng [{min_val}, {max_val}])")
                
                # Ví dụ log mẫu để kiểm tra
                # sample_bad = df.loc[mask_outlier, col].iloc[0]
                # print(f"       -> Ví dụ giá trị lỗi: {sample_bad}")

    print(f"  RESULT: Tổng cộng {total_removed} giá trị vô lý đã được chuyển thành NaN")
    return df_out


def load_city_data(city_folder, city_code):
    """
    Tải dữ liệu chất lượng không khí và thời tiết từ file CSV.

    Parameters:
    -----------
    city_folder : str
        Tên thư mục chứa các file dữ liệu
    city_code : str
        Mã thành phố chuẩn hóa (CT, HCM, HN)

    Returns:
    --------
    tuple
        (air_quality_df, weather_df) hoặc (None, None) khi lỗi
    """
    air_path = f'data/{city_folder}/air_quality_{city_code}.csv'
    weather_path = f'data/{city_folder}/weather_hourly_{city_code}.csv'

    try:
        df_air = pd.read_csv(air_path)
        df_weather = pd.read_csv(weather_path)
        print(f"  TẢI DỮ LIỆU THÀNH CÔNG")
        print(f"     Bộ dữ liệu chất lượng không khí: {df_air.shape[0]} hàng × {df_air.shape[1]} cột")
        print(f"     Bộ dữ liệu thời tiết: {df_weather.shape[0]} hàng × {df_weather.shape[1]} cột")
        return df_air, df_weather
    except FileNotFoundError as e:
        print(f"  ERROR: {e}")
        return None, None


print("HÀM HỖ TRỢ ĐÃ ĐĂNG KÝ:")
print("   - remove_frozen_days(): Phát hiện và loại bỏ cảm biến đóng băng")
print("   - load_city_data(): Tải file dữ liệu với xác thực")

HÀM HỖ TRỢ ĐÃ ĐĂNG KÝ:
   - remove_frozen_days(): Phát hiện và loại bỏ cảm biến đóng băng
   - load_city_data(): Tải file dữ liệu với xác thực


---

##  4: Xử Lý Timestamp & Loại Bỏ Trùng Lặp

### Chuẩn Hóa Dữ Liệu Thời Gian

**Mục Tiêu**:
1. Phân tích chuỗi timestamp thành đối tượng datetime
2. Chuẩn hóa tên cột timestamp trên các bộ dữ liệu
3. Loại bỏ các bản ghi thời gian trùng lặp
4. Thiết lập chỉ mục thời gian cho các thao tác chuỗi thời gian

### Chiến Lược Xử Lý Trùng Lặp

Khi phát hiện timestamp trùng lặp:
- **Sắp Xếp**: Các bản ghi được sắp xếp theo thứ tự thời gian
- **Loại Trùng**: Lần xuất hiện đầu tiên được giữ lại ('keep=first')
- **Lý Do**: Giả định các bản ghi sớm hơn đáng tin cậy hơn (ít có khả năng là hiệu chỉnh hồi tố)

### Định Dạng Timestamp

Định dạng đầu vào: `YYYY-MM-DD:HH` (ví dụ: "2023-01-07:17")

Trình phân tích xử lý các biến thể định dạng sử dụng `errors='coerce'` để ngăn lỗi quy trình khi gặp ngày không hợp lệ.

In [4]:
def preprocess_timestamps(df_air, df_weather, city_folder):
    """
    Chuẩn hóa định dạng timestamp và loại bỏ trùng lặp thời gian.

    Hàm này xử lý các quy ước đặt tên cột timestamp khác nhau
    và đảm bảo đánh chỉ mục thời gian nhất quán trên các bộ dữ liệu.

    Parameters:
    -----------
    df_air : pd.DataFrame
        Đo lường chất lượng không khí
    df_weather : pd.DataFrame
        Quan sát khí tượng
    city_folder : str
        Mã định danh thành phố cho thông báo lỗi

    Returns:
    --------
    tuple
        (air_df_đã_xử_lý, weather_df_đã_xử_lý) với chỉ mục datetime
    """
    # Store a list of columns to be dropped after processing
    cols_to_drop_air = []

    # Identify the primary timestamp column for air quality data, prioritizing 'timestamp_local'
    timestamp_col_to_parse = None

    if 'timestamp_local' in df_air.columns:
        timestamp_col_to_parse = 'timestamp_local'
        # If 'datetime' also exists, it should be dropped as 'timestamp_local' is preferred.
        if 'datetime' in df_air.columns:
            cols_to_drop_air.append('datetime')
            print("     Đã đánh dấu cột 'datetime' để loại bỏ vì 'timestamp_local' được ưu tiên.")
    elif 'datetime' in df_air.columns:
        timestamp_col_to_parse = 'datetime'
    else:
        print(f"  ERROR: Không tìm thấy cột timestamp hợp lệ trong dữ liệu chất lượng không khí cho {city_folder}")
        return None, None

    # Phân tích timestamp với xử lý lỗi (đã loại bỏ format để tự động nhận dạng)
    df_air[timestamp_col_to_parse] = pd.to_datetime(
        df_air[timestamp_col_to_parse],
        errors='coerce' # Giữ errors='coerce' để xử lý các giá trị không thể phân tích cú pháp
    )

    # Diagnostic prints
    print(f"     Kiểm tra cột timestamp_local trong dữ liệu chất lượng không khí sau khi chuyển đổi:")
    print(f"        Dtype: {df_air[timestamp_col_to_parse].dtype}")
    if not df_air.empty:
        print(f"        Giá trị đầu tiên (đã chuyển đổi): {df_air.iloc[0][timestamp_col_to_parse]}")
    else:
        print("        DataFrame rỗng sau khi chuyển đổi timestamp.")
    print(f"        Số lượng giá trị NaT: {df_air[timestamp_col_to_parse].isna().sum()}")
    print(f"        Số lượng timestamp duy nhất: {df_air[timestamp_col_to_parse].nunique()}")

    # Đổi tên 'datetime' thành 'timestamp_local' nếu 'datetime' đã được sử dụng
    if timestamp_col_to_parse == 'datetime':
        df_air.rename(columns={'datetime': 'timestamp_local'}, inplace=True)
        print("     Đã đổi tên cột 'datetime' thành 'timestamp_local' cho dữ liệu chất lượng không khí.")

    # Loại bỏ các cột timestamp dư thừa
    if cols_to_drop_air:
        df_air.drop(columns=cols_to_drop_air, inplace=True)
        print(f"     Đã loại bỏ các cột dư thừa từ dữ liệu chất lượng không khí: {', '.join(cols_to_drop_air)}")

    # Phân tích timestamp thời tiết
    df_weather['timestamp_local'] = pd.to_datetime(df_weather['timestamp_local'])

    # Loại bỏ trùng lặp với thống kê
    air_before = len(df_air)
    weather_before = len(df_weather)

    df_air = df_air.sort_values('timestamp_local').drop_duplicates(
        subset=['timestamp_local'],
        keep='first'
    )
    df_weather = df_weather.sort_values('timestamp_local').drop_duplicates(
        subset=['timestamp_local'],
        keep='first'
    )

    print(f"  LOẠI BỎ TRÙNG LẶP:")
    print(f"     Air quality: {air_before - len(df_air)} trùng lặp đã loại bỏ")
    print(f"     Weather: {weather_before - len(df_weather)} trùng lặp đã loại bỏ")

    return df_air, df_weather


print("HÀM XỬ LÝ TIMESTAMP ĐÃ ĐƯỢC ĐỊNH NGHĨA")

HÀM XỬ LÝ TIMESTAMP ĐÃ ĐƯỢC ĐỊNH NGHĨA


---

##  5: Kết Hợp Dữ Liệu Dựa Trên Timestamp Gốc (V5)

### Phương Pháp Merge Trực Tiếp

**Thay Đổi So Với V4**:
- **V4**: Sử dụng `reindex()` + `concat()` - tạo full time grid rồi concat
- **V5**: Sử dụng `pd.merge()` - merge trực tiếp trên cột `timestamp_local` gốc

**Chiến Lược V5**:
1. **Feature Selection**: Extract only specified biến số from raw datasets
2. **Direct Merge**: Merge hai dataframe dựa trên cột timestamp_local gốc (OUTER JOIN)
3. **Reindex**: Sau khi merge, mới reindex theo full time grid
4. **Set Index**: Cuối cùng set timestamp_local làm index

### Lợi Ích

**Ưu Điểm**:
- **Chính Xác Hơn**: Merge dựa trên timestamp thực tế có trong data
- **Linh Hoạt**: Dễ debug và kiểm tra dữ liệu sau merge
- **Rõ Ràng**: Logic merge rõ ràng hơn

**So Sánh**:
- Cả hai phương pháp đều tạo ra kết quả cuối cùng tương tự
- V5 rõ ràng hơn về cách 2 file được merge với nhau


In [5]:
def merge_on_timestamp(df_air, df_weather, full_time_index):
    """
    Merge dữ liệu air quality và weather dựa trên cột timestamp_local gốc.

    V5 CHANGES:
    - Sử dụng pd.merge() thay vì reindex() + concat()
    - Merge trực tiếp trên timestamp_local column

    Parameters:
    -----------
    df_air : pd.DataFrame
        Air quality data (timestamp_local as column)
    df_weather : pd.DataFrame
        Weather data (timestamp_local as column)
    full_time_index : pd.DatetimeIndex
        Full time grid for reindexing after merge

    Returns:
    --------
    tuple
        (df_merged, valid_air_cols, valid_weather_cols)
    """
    # Filter chỉ các features cần thiết
    valid_air_cols = [c for c in feature_air if c in df_air.columns]
    valid_weather_cols = [c for c in feature_weather if c in df_weather.columns]

    # Select columns + timestamp_local
    df_air_selected = df_air[["timestamp_local"] + valid_air_cols].copy()
    df_weather_selected = df_weather[["timestamp_local"] + valid_weather_cols].copy()

    print(f"  LỰA CHỌN ĐẶC TRƯNG:")
    print(f"     Air quality: {len(valid_air_cols)} biến số - {valid_air_cols}")
    print(f"     Weather: {len(valid_weather_cols)} biến số - {valid_weather_cols}")

    # THAY ĐỔI CHÍNH V5: Merge trên timestamp_local gốc
    df_merged = pd.merge(
        df_air_selected,
        df_weather_selected,
        on="timestamp_local",
        how="outer",  # Outer join để giữ tất cả timestamps
        suffixes=("_air", "_weather")
    )

    print(f"  MERGE HOÀN TẤT:")
    print(f"     Số dòng sau merge: {len(df_merged):,}")
    print(f"     Air records: {len(df_air_selected):,}")
    print(f"     Weather records: {len(df_weather_selected):,}")

    # Set index timestamp_local
    df_merged.set_index("timestamp_local", inplace=True)

    # Reindex theo full time grid
    df_merged = df_merged.reindex(full_time_index)

    print(f"  ĐÁNH CHỈ MỤC THỜI GIAN: Mở rộng đến {len(full_time_index):,} quan sát theo giờ")
    print(f"  BỘ DỮ LIỆU CUỐI: {df_merged.shape[0]} hàng × {df_merged.shape[1]} cột")
    print(f"  GIÁ TRỊ THIẾU: {df_merged.isna().sum().sum():,} mục NaN ({df_merged.isna().sum().sum() / df_merged.size * 100:.2f}%)")

    return df_merged, valid_air_cols, valid_weather_cols


print("HÀM MERGE DỰA TRÊN TIMESTAMP ĐÃ ĐƯỢC ĐỊNH NGHĨA (V5)")

HÀM MERGE DỰA TRÊN TIMESTAMP ĐÃ ĐƯỢC ĐỊNH NGHĨA (V5)


---

##  6: Kỹ Thuật Đặc Trưng

### Tạo Đặc Trưng Dẫn Xuất

#### 1. Phân Tích Hướng Gió

**Vấn Đề**: Hướng gió (0-360°) là biến vòng tròn - 0° và 360° giống nhau

**Giải Pháp**: Biến đổi lượng giác thành các thành phần Descartes:
```
wind_sin = sin(θ)  # Thành phần Bắc-Nam
wind_cos = cos(θ)  # Thành phần Đông-Tây
```

**Lợi Ích**:
- Loại bỏ sự gián đoạn tại ranh giới 0°/360°
- Preserves directional information in two continuous biến số
- Cải thiện hiệu suất mô hình học máy

#### 2. Mã Hóa Chu Kỳ Ngày

**Biến Đổi**: Biến phân loại 'd' (ngày) / 'n' (đêm) → Nhị phân 1/0

**Lý Do**: Cho phép các phép toán số học trong khi vẫn bảo toàn thông tin chu kỳ ngày đêm

### Chuẩn Hóa Kiểu Dữ Liệu

Tất cả các cột số được chuyển đổi sang kiểu số với xử lý lỗi (`errors='coerce'`) để ngăn lỗi xử lý liên quan đến kiểu dữ liệu.

In [6]:
def feature_engineering(df_merged, valid_air_cols, valid_weather_cols):
    """
    Tạo các đặc trưng dẫn xuất và chuẩn hóa kiểu dữ liệu.

    Các biến đổi chính:
    1. Phân tích hướng gió vòng tròn thành các thành phần Descartes
    2. Mã hóa biến chu kỳ ngày phân loại
    3. Đảm bảo tất cả đặc trưng có kiểu số phù hợp

    Parameters:
    -----------
    df_merged : pd.DataFrame
        Dữ liệu chất lượng không khí và thời tiết đã kết hợp
    valid_air_cols : list
        Tên đặc trưng chất lượng không khí
    valid_weather_cols : list
        Tên đặc trưng thời tiết

    Returns:
    --------
    tuple
        (df_nâng_cao, tất_cả_cột_số)
    """
    print("  KỸ THUẬT ĐẶC TRƯNG:")

    # Chuyển chu kỳ ngày sang số nhị phân
    if 'pod' in df_merged.columns:
        if df_merged['pod'].dtype == 'object':
            df_merged['pod'] = df_merged['pod'].map({'d': 1, 'n': 0})
        df_merged['pod'] = pd.to_numeric(df_merged['pod'], errors='coerce')
        print("     Chu kỳ ngày đã mã hóa: ngày=1, đêm=0")

    # Phân tích hướng gió thành các thành phần Descartes
    if 'wind_dir' in df_merged.columns and 'wind_spd' in df_merged.columns:
        df_merged['wind_dir'] = pd.to_numeric(df_merged['wind_dir'], errors='coerce')
        rad = np.deg2rad(df_merged['wind_dir'])
        df_merged['wind_sin'] = np.sin(rad)
        df_merged['wind_cos'] = np.cos(rad)
        print("     Hướng gió đã phân tích: wind_sin, wind_cos đã tạo")

    # Compile all numeric cột
    all_known_num_cols = list(set(valid_air_cols + valid_weather_cols))
    if 'wind_sin' in df_merged.columns:
        all_known_num_cols.append('wind_sin')
    if 'wind_cos' in df_merged.columns:
        all_known_num_cols.append('wind_cos')

    # Chuẩn hóa kiểu số
    for col in all_known_num_cols:
        if col in df_merged.columns:
            df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')

    print(f"     Chuẩn hóa kiểu dữ liệu: {len(all_known_num_cols)} numeric cột")

    return df_merged, all_known_num_cols


print("HÀM KỸ THUẬT ĐẶC TRƯNG ĐÃ ĐƯỢC ĐỊNH NGHĨA")

HÀM KỸ THUẬT ĐẶC TRƯNG ĐÃ ĐƯỢC ĐỊNH NGHĨA


---

##  7: Điền Khuyết Giá Trị

### Chiến Lược Điền Khuyết Kết Hợp

#### Giai đoạn 1: Nội Suy Tuyến Tính Theo Thời Gian

**Phương Pháp**: Nội suy tuyến tính nhận biết thời gian

**Ứng Dụng**: Khoảng trống nhỏ (≤ 6 giờ liên tiếp)

**Lý Do**:
- Environmental biến số exhibit temporal autocorrelation
- Khoảng trống ngắn có thể đại diện cho lỗi truyền thông cảm biến hơn là suy giảm
- Nội suy tuyến tính bảo toàn xu hướng thời gian
- Hiệu quả tính toán cho dữ liệu tần số cao

**Hạn Chế**: Không thể điền khoảng trống > 6 giờ (tham số: `limit=6`)

#### Giai đoạn 2: Điền Khuyết K-NN

**Phương Pháp**: Điền khuyết KNN với trọng số khoảng cách

**Tham Số**:
- `n_neighbors=12`: Sử dụng 12 quan sát tương tự nhất
- `weights='distance'`: Trọng số khoảng cách nghịch đảo cho đóng góp của láng giềng

**Ứng Dụng**: Khoảng trống lớn và mẫu thiếu phức tạp

**Ưu Điểm**:
- Nắm bắt mối quan hệ giữa các đặc trưng (ví dụ: tương quan nhiệt độ-độ ẩm)
- Bền vững với khoảng trống dữ liệu dài
- Trọng số khoảng cách ngăn chặn thiên lệch từ láng giềng xa

**Tiền Xử Lý**: Các cột có 100% dữ liệu thiếu được loại trừ để ngăn lỗi điền khuyết

### Chỉ Số Hiệu Suất Điền Khuyết

Hàm báo cáo:
1. Số lượng giá trị thiếu ban đầu
2. Giá trị được điền bởi nội suy tuyến tính
3. Giá trị được điền bởi KNN
4. Tổng phạm vi điền khuyết

In [7]:
def impute_missing_values(df_merged):
    """
    Điền giá trị thiếu sử dụng phương pháp điền khuyết kết hợp.

    Chiến lược hai giai đoạn:
    1. Nội suy tuyến tính theo thời gian cho khoảng trống nhỏ (≤6h)
    2. K-Láng Giềng Gần Nhất cho khoảng trống lớn và mẫu phức tạp

    Parameters:
    -----------
    df_merged : pd.DataFrame
        Bộ dữ liệu với giá trị thiếu

    Returns:
    --------
    pd.DataFrame
        Bộ dữ liệu đã điền khuyết hoàn toàn
    """
    print("  ĐIỀN KHUYẾT GIÁ TRỊ:")

    # Giai đoạn 1: Nội suy tuyến tính theo thời gian
    original_nans = df_merged.isna().sum().sum()
    print(f"     Giá trị thiếu ban đầu: {original_nans:,} ({original_nans / df_merged.size * 100:.2f}%)")

    df_merged = df_merged.interpolate(method='time', limit=6)
    after_linear = df_merged.isna().sum().sum()
    filled_by_linear = original_nans - after_linear
    print(f"     Giai đoạn 1 (Tuyến tính): {filled_by_linear:,} giá trị đã điền")

    # Giai đoạn 2: Điền khuyết KNN cho các khoảng trống còn lại
    num_cols = df_merged.select_dtypes(include=[np.number]).columns
    df_for_knn = df_merged[num_cols]

    # Remove entirely missing cột
    cols_to_drop = df_for_knn.columns[df_for_knn.isnull().all()]
    if len(cols_to_drop) > 0:
        print(f"     CẢNH BÁO: {len(cols_to_drop)} cột entirely missing: {list(cols_to_drop)}")
        df_for_knn = df_for_knn.drop(columns=cols_to_drop)

    # Áp dụng điền khuyết KNN
    imputer = KNNImputer(n_neighbors=12, weights='distance')
    df_imputed_val = imputer.fit_transform(df_for_knn)
    df_final = pd.DataFrame(
        df_imputed_val,
        columns=df_for_knn.columns,
        index=df_merged.index
    )

    filled_by_knn = after_linear - df_final.isna().sum().sum()
    print(f"     Giai đoạn 2 (KNN): {filled_by_knn:,} giá trị đã điền")
    print(f"     TỔNG ĐÃ ĐIỀN: {filled_by_linear + filled_by_knn:,} values ({(filled_by_linear + filled_by_knn) / original_nans * 100:.1f}% phạm vi)")

    return df_final


print("HÀM ĐIỀN KHUYẾT ĐÃ ĐƯỢC ĐỊNH NGHĨA")

HÀM ĐIỀN KHUYẾT ĐÃ ĐƯỢC ĐỊNH NGHĨA


---

##  8: Kiểm Định Chất Lượng & Xuất Dữ Liệu

### Kiểm Tra Đảm Bảo Chất Lượng

#### 1. Hiệu Chỉnh Chu Kỳ Ngày

**Quy Tắc**: Ban ngày = 06:00-18:00 (bao gồm)

**Lý Do**: Imputation may introduce inconsistent POD values; this ensures temporal consistency

#### 2. Ràng Buộc Giá Trị Không Âm

**Các Biến Bị Ảnh Hưởng**:
- **Chất Lượng Không Khí**: AQI, CO, NO₂, O₃, PM10, PM2.5, SO₂ (nồng độ không thể âm)
- **Thời Tiết**: Độ ẩm tương đối, Áp suất, Tốc độ gió, Độ che phủ mây, Lượng mưa

**Hành Động**: Giá trị âm được cắt về 0

**Lý Do**: KNN imputation may produce physically impossible giá trị âm; correction ensures data validity

#### 3. Độ Chính Xác Số Học

**Làm Tròn**: Tất cả giá trị được làm tròn đến 2 chữ số thập phân

**Mục Đích**:
- Giảm kích thước file
- Loại bỏ độ chính xác giả từ giá trị điền khuyết
- Đảm bảo tính nhất quán giữa các bộ dữ liệu

### Đặc Tả Xuất Dữ Liệu

**Định Dạng Đầu Ra**: CSV 

**Chỉ Mục**: timestamp_local (chỉ mục thời gian được bảo toàn)

**Vị Trí**: `data/cleaned_data_{CITY_CODE}.csv`

In [8]:
def sanity_check_and_export(df_final, city_code):
    """
    Kiểm định tính toàn vẹn dữ liệu và xuất ra CSV.

    Các bước đảm bảo chất lượng:
    1. Hiệu chỉnh chu kỳ ngày dựa trên giờ
    2. Enforce non-negativity constraints for physical biến số
    3. Chuẩn hóa độ chính xác số học
    4. Xuất ra file CSV có phiên bản

    Parameters:
    -----------
    df_final : pd.DataFrame
        Bộ dữ liệu đã xử lý hoàn toàn
    city_code : str
        Mã định danh thành phố (CT, HCM, HN)

    Returns:
    --------
    pd.DataFrame
        Bộ dữ liệu đã kiểm định và xuất
    """
    print("  KIỂM ĐỊNH DỮ LIỆU:")

    # Hiệu chỉnh chu kỳ ngày dựa trên giờ (06:00-18:00 = day)
    if 'pod' in df_final.columns:
        df_final['pod'] = np.where(
            (df_final.index.hour >= 6) & (df_final.index.hour <= 18),
            1, 0
        )
        print("     Chu kỳ ngày được tính lại (06:00-18:00 = ngày)")

    # Enforce non-negativity for physical biến số
    non_negative_cols = [
        'aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2',
        'rh', 'pres', 'wind_spd', 'clouds', 'precip'
    ]

    total_negatives = 0
    for col in non_negative_cols:
        if col in df_final.columns:
            neg_count = (df_final[col] < 0).sum()
            if neg_count > 0:
                df_final.loc[df_final[col] < 0, col] = 0
                total_negatives += neg_count
                print(f"     {col}: {neg_count} giá trị âm đã hiệu chỉnh về 0")

    if total_negatives > 0:
        print(f"     Tổng hiệu chỉnh: {total_negatives} giá trị âm")

    # Chuẩn hóa độ chính xác số học
    df_final = df_final.round(2)
    print("     Độ chính xác số học: chuẩn hóa đến 2 chữ số thập phân")

    # Xuất ra CSV
    out_path = f'data/cleaned_data_{city_code}.csv'
    df_final.to_csv(out_path, index_label='timestamp_local')

    print(f"\n  TÓM TẮT XUẤT:")
    print(f"     Đường dẫn file: {out_path}")
    print(f"     Kích thước: {df_final.shape[0]:,} quan sát × {df_final.shape[1]} đặc trưng")
    print(f"     Tổng điểm dữ liệu: {df_final.shape[0] * df_final.shape[1]:,}")
    print(f"     Temporal phạm vi: {df_final.index[0]} to {df_final.index[-1]}")

    return df_final


print("HÀM KIỂM ĐỊNH & XUẤT ĐÃ ĐƯỢC ĐỊNH NGHĨA")

HÀM KIỂM ĐỊNH & XUẤT ĐÃ ĐƯỢC ĐỊNH NGHĨA


---

##  9: Pipeline xử lí

### Kiến Trúc Quy Trình

Hàm này điều phối quy trình xử lý dữ liệu hoàn chỉnh:

```
ĐẦU VÀO: Các file CSV thô (chất lượng không khí + thời tiết)
  ↓
[1] Load Data
  ↓
[2] Timestamp Processing & Deduplication
  ↓
[3] Temporal Merging & Reindexing
  ↓
[4] Feature Engineering
  ↓
[5] Frozen Data Removal
  ↓
[6] Hybrid Imputation (Linear + KNN)
  ↓
[7] Validation & Export
  ↓
ĐẦU RA: File CSV sạch + DataFrame đã xử lý
```

### Đặc Điểm Quy Trình

- **Thiết Kế Mô-đun**: Mỗi bước có thể kiểm tra độc lập
- **Xử Lý Lỗi**: Lỗi trả về None với thông báo chi tiết
- **Theo Dõi Tiến trình**: Ghi log chi tiết ở mỗi giai đoạn
- **Xem Trước Dữ Liệu**: Hiển thị mẫu đầu ra để kiểm tra trực quan

### Cách Sử Dụng

```python
df = processing_pipeline('Hanoi', 'HN')
```

In [9]:
def processing_pipeline(city_folder, city_code):
    """
    Thực thi quy trình tiền xử lý dữ liệu hoàn chỉnh cho một thành phố.

    Hàm này tích hợp tất cả các bước tiền xử lý vào một quy trình thống nhất,
    từ tải CSV thô đến tạo đầu ra đã kiểm định.

    Parameters:
    -----------
    city_folder : str
        Tên thư mục (Cantho, HCM, Hanoi)
    city_code : str
        Mã thành phố chuẩn hóa (CT, HCM, HN)

    Returns:
    --------
    pd.DataFrame or None
        Bộ dữ liệu đã xử lý hoàn toàn or None on failure
    """
    print(f"\n{'='*80}")
    print(f"BẮT ĐẦU XỬ LÝ: {city_folder} ({city_code})")
    print(f"{'='*80}")

    # Bước 1: Tải Dữ Liệu
    print("\n[BƯỚC 1/7] TẢI DỮ LIỆU")
    df_air, df_weather = load_city_data(city_folder, city_code)
    if df_air is None or df_weather is None:
        return None

    # Bước 2: Xử Lý Timestamp
    print("\n[BƯỚC 2/7] CHUẨN HÓA TIMESTAMP")
    df_air, df_weather = preprocess_timestamps(df_air, df_weather, city_folder)
    if df_air is None or df_weather is None:
        return None

    # Bước 3: Kết Hợp & Đánh Chỉ Mục
    print("\n[BƯỚC 3/7] CĂN CHỈNH THỜI GIAN & KẾT HỢP")
    df_merged, valid_air_cols, valid_weather_cols = merge_on_timestamp(
        df_air, df_weather, full_time_index
    )

    # Bước 4: Kỹ Thuật Đặc Trưng
    print("\n[BƯỚC 4/7] KỸ THUẬT ĐẶC TRƯNG")
    df_merged, all_num_cols = feature_engineering(
        df_merged, valid_air_cols, valid_weather_cols
    )

    # Bước 5: Loại Bỏ Dữ Liệu Đóng Băng
    print("\n[BƯỚC 5.1/7] PHÁT HIỆN DỮ LIỆU ĐÓNG BĂNG")
    frozen_check_cols = [c for c in all_num_cols if c not in ['pod', 'precip']]
    df_merged = remove_frozen_days(df_merged, frozen_check_cols)
    
    print("\n[BƯỚC 5.2/7] LOẠI BỎ GIÁ TRỊ PHI THỰC TẾ (PHYSICAL OUTLIERS)")
    df_merged = remove_physical_outliers(df_merged, city_code)

    # Bước 6: Điền Khuyết
    print("\n[BƯỚC 6/7] ĐIỀN KHUYẾT GIÁ TRỊ")
    df_final = impute_missing_values(df_merged)

    # Bước 7: Kiểm Định & Xuất
    print("\n[BƯỚC 7/7] KIỂM ĐỊNH & XUẤT")
    df_final = sanity_check_and_export(df_final, city_code)

    # Hiển thị mẫu
    print("\nMẪU ĐẦU RA (3 quan sát đầu tiên):")
    display(df_final.head(3))

    print(f"\n{'='*80}")
    print(f"XỬ LÝ HOÀN TẤT: {city_folder} ({city_code})")
    print(f"{'='*80}\n")

    return df_final


print("QUY TRÌNH XỬ LÝ HOÀN CHỈNH ĐÃ ĐƯỢC ĐỊNH NGHĨA")

QUY TRÌNH XỬ LÝ HOÀN CHỈNH ĐÃ ĐƯỢC ĐỊNH NGHĨA


---

##  10: Thực Thi Xử Lý Hàng Loạt

### Xử Lý Đa Thành Phố

Cell này thực thi quy trình tuần tự cho cả ba thành phố:

1. **Can Tho (CT)** - Đại diện đồng bằng sông Cửu Long
2. **Ho Chi Minh City (HCM)** - Siêu đô thị phía Nam
3. **Hanoi (HN)** - Thủ đô phía Bắc

### Khả Năng Chống Lỗi

Quy trình hàng loạt chịu lỗi:
- Lỗi ở một thành phố không dừng xử lý các thành phố khác
- Các thành phố lỗi được ghi lại với giá trị None
- Xử lý tiếp tục đến hoàn thành

### Lưu Trữ Kết Quả

Kết quả được lưu trong dictionary `results = {tên_thành_phố: dataframe_đã_xử_lý}`

Điều này cho phép:
- Truy cập riêng lẻ vào bộ dữ liệu thành phố
- Phân tích hoặc so sánh tiếp theo
- Tạo thống kê tổng hợp

In [10]:
# Thực thi quy trình cho tất cả các thành phố
print("="*80)
print("XỬ LÝ HÀNG LOẠT: TẤT CẢ CÁC THÀNH PHỐ")
print("="*80)

results = {}
for city_folder, city_code in cities_config.items():
    results[city_folder] = processing_pipeline(city_folder, city_code)

XỬ LÝ HÀNG LOẠT: TẤT CẢ CÁC THÀNH PHỐ

BẮT ĐẦU XỬ LÝ: Cantho (CT)

[BƯỚC 1/7] TẢI DỮ LIỆU
  TẢI DỮ LIỆU THÀNH CÔNG
     Bộ dữ liệu chất lượng không khí: 22504 hàng × 11 cột
     Bộ dữ liệu thời tiết: 22368 hàng × 29 cột

[BƯỚC 2/7] CHUẨN HÓA TIMESTAMP
     Đã đánh dấu cột 'datetime' để loại bỏ vì 'timestamp_local' được ưu tiên.
     Kiểm tra cột timestamp_local trong dữ liệu chất lượng không khí sau khi chuyển đổi:
        Dtype: datetime64[ns]
        Giá trị đầu tiên (đã chuyển đổi): 2023-01-08 00:00:00
        Số lượng giá trị NaT: 0
        Số lượng timestamp duy nhất: 22502
     Đã loại bỏ các cột dư thừa từ dữ liệu chất lượng không khí: datetime
  LOẠI BỎ TRÙNG LẶP:
     Air quality: 2 trùng lặp đã loại bỏ
     Weather: 0 trùng lặp đã loại bỏ

[BƯỚC 3/7] CĂN CHỈNH THỜI GIAN & KẾT HỢP
  LỰA CHỌN ĐẶC TRƯNG:
     Air quality: 7 biến số - ['aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
     Weather: 9 biến số - ['temp', 'rh', 'pres', 'wind_spd', 'wind_dir', 'clouds', 'precip', 'pod

,aqi,co,no2,o3,pm10,pm25,so2,temp,rh,pres,wind_spd,wind_dir,clouds,precip,pod,dewpt,wind_sin,wind_cos
2023-01-01 00:00:00,152.0,432.8,57.3,22.3,83.3,56.67,37.7,22.0,83.0,1015.0,1.0,320.0,100.0,0.25,0,19.0,-0.64,0.77
2023-01-01 01:00:00,150.0,402.1,54.0,20.0,79.0,54.00,32.0,22.0,83.0,1014.0,2.1,330.0,99.0,0.00,0,19.0,-0.50,0.87
2023-01-01 02:00:00,134.0,361.2,45.3,24.3,70.3,48.00,31.7,22.0,83.0,1013.0,2.1,330.0,100.0,0.00,0,19.0,-0.50,0.87



XỬ LÝ HOÀN TẤT: Cantho (CT)


BẮT ĐẦU XỬ LÝ: HCM (HCM)

[BƯỚC 1/7] TẢI DỮ LIỆU
  TẢI DỮ LIỆU THÀNH CÔNG
     Bộ dữ liệu chất lượng không khí: 25597 hàng × 11 cột
     Bộ dữ liệu thời tiết: 25560 hàng × 29 cột

[BƯỚC 2/7] CHUẨN HÓA TIMESTAMP
     Đã đánh dấu cột 'datetime' để loại bỏ vì 'timestamp_local' được ưu tiên.
     Kiểm tra cột timestamp_local trong dữ liệu chất lượng không khí sau khi chuyển đổi:
        Dtype: datetime64[ns]
        Giá trị đầu tiên (đã chuyển đổi): 2023-02-01 00:00:00
        Số lượng giá trị NaT: 0
        Số lượng timestamp duy nhất: 25561
     Đã loại bỏ các cột dư thừa từ dữ liệu chất lượng không khí: datetime
  LOẠI BỎ TRÙNG LẶP:
     Air quality: 36 trùng lặp đã loại bỏ
     Weather: 0 trùng lặp đã loại bỏ

[BƯỚC 3/7] CĂN CHỈNH THỜI GIAN & KẾT HỢP
  LỰA CHỌN ĐẶC TRƯNG:
     Air quality: 7 biến số - ['aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
     Weather: 9 biến số - ['temp', 'rh', 'pres', 'wind_spd', 'wind_dir', 'clouds', 'precip', 'pod', 'dewpt

,aqi,co,no2,o3,pm10,pm25,so2,temp,rh,pres,wind_spd,wind_dir,clouds,precip,pod,dewpt,wind_sin,wind_cos
2023-01-01 00:00:00,169.0,320.8,79.3,24.3,100.7,69.33,141.0,27.65,88.49,1009.73,1.94,258.95,50.71,0.14,0,25.48,-0.76,0.08
2023-01-01 01:00:00,155.0,280.1,68.0,27.0,86.0,59.00,119.0,27.63,85.52,1009.70,2.08,253.28,61.50,0.21,0,24.95,-0.56,0.28
2023-01-01 02:00:00,153.0,279.2,64.7,24.3,80.3,55.00,114.3,27.12,84.86,1009.85,2.10,266.87,65.11,0.27,0,24.27,-0.74,0.27



XỬ LÝ HOÀN TẤT: HCM (HCM)


BẮT ĐẦU XỬ LÝ: Hanoi (HN)

[BƯỚC 1/7] TẢI DỮ LIỆU
  TẢI DỮ LIỆU THÀNH CÔNG
     Bộ dữ liệu chất lượng không khí: 25597 hàng × 11 cột
     Bộ dữ liệu thời tiết: 25560 hàng × 29 cột

[BƯỚC 2/7] CHUẨN HÓA TIMESTAMP
     Đã đánh dấu cột 'datetime' để loại bỏ vì 'timestamp_local' được ưu tiên.
     Kiểm tra cột timestamp_local trong dữ liệu chất lượng không khí sau khi chuyển đổi:
        Dtype: datetime64[ns]
        Giá trị đầu tiên (đã chuyển đổi): 2023-02-01 00:00:00
        Số lượng giá trị NaT: 0
        Số lượng timestamp duy nhất: 25561
     Đã loại bỏ các cột dư thừa từ dữ liệu chất lượng không khí: datetime
  LOẠI BỎ TRÙNG LẶP:
     Air quality: 36 trùng lặp đã loại bỏ
     Weather: 0 trùng lặp đã loại bỏ

[BƯỚC 3/7] CĂN CHỈNH THỜI GIAN & KẾT HỢP
  LỰA CHỌN ĐẶC TRƯNG:
     Air quality: 7 biến số - ['aqi', 'co', 'no2', 'o3', 'pm10', 'pm25', 'so2']
     Weather: 9 biến số - ['temp', 'rh', 'pres', 'wind_spd', 'wind_dir', 'clouds', 'precip', 'pod', 'dewpt'

,aqi,co,no2,o3,pm10,pm25,so2,temp,rh,pres,wind_spd,wind_dir,clouds,precip,pod,dewpt,wind_sin,wind_cos
2023-01-01 00:00:00,155.0,224.5,9.0,55.7,73.8,59.0,62.3,17.55,77.15,1018.34,1.61,173.74,51.03,0.20,0,13.20,0.50,-0.10
2023-01-01 01:00:00,171.0,206.0,6.0,56.0,88.8,71.0,59.0,17.61,83.74,1019.14,1.52,149.41,64.95,0.12,0,14.70,0.14,0.17
2023-01-01 02:00:00,179.0,203.7,6.0,54.0,96.3,77.0,58.0,17.80,81.77,1020.38,1.16,155.49,66.77,0.06,0,14.46,0.12,0.41



XỬ LÝ HOÀN TẤT: Hanoi (HN)



---

##  11: Tổng Kết Cuối Cùng & Báo Cáo Thống Kê


#### 1. Đặc Điểm Bộ Dữ Liệu
- **Kích Thước**: Số lượng quan sát và đặc trưng
- **Phạm Vi Thời Gian**: Timestamp đầu tiên và cuối cùng
- **Feature List**: All biến số in final dataset

#### 2. Thống Kê Mô Tả

Cho mỗi thành phố, báo cáo bao gồm:
- **Số Lượng**: Quan sát không thiếu cho mỗi biến
- **Trung Bình**: Giá trị trung bình trong khoảng thời gian
- **Min/Max**: Phạm vi giá trị quan sát được


In [24]:
print("\n" + "="*80)
print("BÁO CÁO XỬ LÝ CUỐI CÙNG")
print("="*80 + "\n")

for city_name, df in results.items():
    if df is not None:
        city_code = cities_config[city_name]
        print(f"\nTHÀNH PHỐ: {city_name} ({city_code})")
        print("-" * 80)
        print(f"   Trạng thái: THÀNH CÔNG")
        print(f"   Kích thước: {df.shape[0]:,} quan sát × {df.shape[1]} đặc trưng")
        print(f"   File đầu ra: data/cleaned_data_{city_code}_v3.csv")
        print(f"   Phạm vi thời gian: {df.index[0]} to {df.index[-1]}")
        print(f"   Danh sách đặc trưng: {', '.join(df.columns.tolist())}")
        print("\n   THỐNG KÊ MÔ TẢ:")
        summary = df.describe().loc[['count', 'mean', 'min', 'max']]
        print(summary.to_string())
        print()
    else:
        print(f"\nTHÀNH PHỐ: {city_name}")
        print("-" * 80)
        print(f"   Trạng thái: XỬ LÝ THẤT BẠI")
        print()

print("="*80)
print("XỬ LÝ HÀNG LOẠT HOÀN TẤT")
print("Tất cả bộ dữ liệu đã làm sạch được xuất với hậu tố phiên bản: _v4")
print("="*80)


BÁO CÁO XỬ LÝ CUỐI CÙNG


THÀNH PHỐ: Cantho (CT)
--------------------------------------------------------------------------------
   Trạng thái: THÀNH CÔNG
   Kích thước: 25,561 quan sát × 18 đặc trưng
   File đầu ra: data/cleaned_data_CT_v3.csv
   Phạm vi thời gian: 2023-01-01 00:00:00 to 2025-12-01 00:00:00
   Danh sách đặc trưng: aqi, co, no2, o3, pm10, pm25, so2, temp, rh, pres, wind_spd, wind_dir, clouds, precip, pod, dewpt, wind_sin, wind_cos

   THỐNG KÊ MÔ TẢ:
                aqi            co           no2            o3          pm10          pm25           so2          temp            rh          pres      wind_spd     wind_dir        clouds        precip           pod         dewpt      wind_sin      wind_cos
count  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.00000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000  25561.000000
mean      58